In [ ]:
!pip3 install imageio

## Importing The Packages

In [ ]:
import numpy as np
import gym
from gym import spaces
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

## Creating The Four Rooms Environment

In [ ]:
class FourRoomsEnv(gym.Env):
    def __init__(self):
        super(FourRoomsEnv, self).__init__()
        self.grid_size = 11
        self.action_space = spaces.Discrete(4)  # 4 actions: up, down, left, right
        self.observation_space = spaces.Discrete(self.grid_size * self.grid_size)
        self.reset()

    def reset(self):
        self.grid = np.zeros((self.grid_size, self.grid_size))
        
        # Define borders and inner walls
        self.grid[:, 0] = -1  # Left border
        self.grid[:, -1] = -1  # Right border
        self.grid[0, :] = -1  # Top border
        self.grid[-1, :] = -1  # Bottom border
        
        # Inner walls
        self.grid[5, 0:5] = -1
        self.grid[5, 6:] = -1
        self.grid[0:5, 5] = -1
        self.grid[6:, 5] = -1
        self.grid[5, 5] = -1
        
        # Open cells in the middle of each wall
        self.grid[5, 2] = 0  # Middle of the left wall
        self.grid[5, 8] = 0  # Middle of the right wall
        self.grid[2, 5] = 0  # Middle of the top wall
        self.grid[8, 5] = 0  # Middle of the bottom wall

        # Define start and goal states
        self.start_state = (1, 1)
        self.goal_state = (9, 9)
        self.state = self.start_state
        return self.state_to_index(self.state)

    def state_to_index(self, state):
        return state[0] * self.grid_size + state[1]

    def index_to_state(self, index):
        row = index // self.grid_size
        col = index % self.grid_size
        return (row, col)

    def step(self, action):
        next_state = list(self.state)
        if action == 0 and next_state[0] > 0:  # Up
            next_state[0] -= 1
        elif action == 1 and next_state[0] < self.grid_size - 1:  # Down
            next_state[0] += 1
        elif action == 2 and next_state[1] > 0:  # Left
            next_state[1] -= 1
        elif action == 3 and next_state[1] < self.grid_size - 1:  # Right
            next_state[1] += 1

        # Check if next state hits a wall (outer borders or inner walls)
        if self.grid[next_state[0], next_state[1]] == -1:
            reward = -5  # Penalty for hitting a wall
            next_state = self.state  # Stay in current state
        else:
            reward = -1  # Default reward for moving

        if next_state == list(self.goal_state):
            reward = 10  # Reward for reaching the goal
            done = True
        else:
            done = False

        self.state = tuple(next_state)
        return self.state_to_index(self.state), reward, done, {}

## Setting The Environment Parameters

In [ ]:
# Parameters for Q-learning
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 1.0  # Initial exploration rate
epsilon_min = 0.1  # Minimum exploration rate
epsilon_decay = 0.995  # Decay rate for exploration probability
num_episodes = 2000  # Increased episodes for better learning
max_steps_per_episode = 100  # Reduced steps per episode

## Initializing the Environment, the Q-table, the epsilon-greedy policy, and the plotting functionality

In [ ]:
# Initialize environment
env = FourRoomsEnv()

# Initialize Q-table
Q = np.zeros((env.observation_space.n, env.action_space.n))

# Epsilon-greedy policy
def epsilon_greedy_policy(state):
    if np.random.rand() < epsilon:
        return env.action_space.sample()  # Explore action space
    else:
        return np.argmax(Q[state])

# Plot grid function
def plot_grid(env):
    agent_pos = env.index_to_state(env.state_to_index(env.state))
    grid = np.copy(env.grid)
    grid[agent_pos[0], agent_pos[1]] = 2  # Mark the agent's position
    grid[env.goal_state[0], env.goal_state[1]] = 3  # Mark the goal state

    plt.figure(figsize=(6, 6))
    plt.imshow(grid, cmap="coolwarm", origin="upper")
    plt.xticks([]), plt.yticks([])
    plt.show()

## Training The Agent Using Q-Learning

In [ ]:
# Training loop with Q-learning
episode_rewards = []
for episode in range(num_episodes):
    state = env.reset()
    done = False
    total_reward = 0

    for step in range(max_steps_per_episode):
        clear_output(wait=True)
        # Visualize the environment
        plot_grid(env)

        # Call The Epsilon Greedy Policy to take an "action"
        # <write your code here!!>
        # Take a step in the environment. (hind: the expected return is the next_state, reward, done, and info)
        # <write your code here!1>

        # Q-learning update
        best_next_action = np.argmax(Q[next_state])
        td_target = reward + gamma * Q[next_state, best_next_action]
        td_error = td_target - Q[state, action]
        
        # Update the Q-value of the current state,action by alpha times the td_error
        #<write your code here!!>

        state = next_state
        total_reward += reward  # Accumulate reward

        if done:
            break

    episode_rewards.append(total_reward)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)  # Decay epsilon

    # Print progress every 5 episodes
    if (episode + 1) % 5 == 0:
        print(f"Episode {episode + 1}/{num_episodes}, Average Reward: {np.mean(episode_rewards[-5:]):.2f}")